# Setup

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

# Build the model

In [2]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)

# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,757,761
Trainable params: 2,757,761
Non-train

# Load the IMDB movie review sentiment data

In [3]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)

print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

17473536/17464789 [==============================] - 1s 0us/step
25000 Training sequences
25000 Validation sequences


In [4]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [5]:
x_val

array([[    0,     0,     0, ...,    14,     6,   717],
       [ 1987, 17975,    45, ...,   125,     4,  3077],
       [ 4468,   189,     4, ...,     9,    57,   975],
       ...,
       [    0,     0,     0, ...,    21,   846,  5518],
       [    0,     0,     0, ...,  2302,     7,   470],
       [    0,     0,     0, ...,    34,  2005,  2643]])

# Train and evaluate the model

In [6]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 [==============================] - 173s 217ms/step - loss: 0.3863 - accuracy: 0.8270 - val_loss: 0.4533 - val_accuracy: 0.8324
Epoch 2/2
782/782 [==============================] - 170s 217ms/step - loss: 0.2023 - accuracy: 0.9250 - val_loss: 0.3597 - val_accuracy: 0.8500
